# 引入计划器（Planner）

计划器是语义内核的基本概念之一。它利用已注册到内核的技能集合，并使用AI，将制定执行给定请求的计划。

点击[这里](https://learn.microsoft.com/zh-cn/semantic-kernel/create-chains/planner)了解更多。

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);

## 设置计划表

计划器位于语义内核的 coresskills 中，需要编排

In [ ]:
// Load native skill into the kernel registry, sharing its functions with prompt templates
var planner = new SequentialPlanner(kernel);

## 提供技能给计划器

计划器需要知道哪些技能是可用的。这里我们将赋予它访问我们在磁盘上定义的 SummarizeSkill 和 WriterSkill 的权限。

In [ ]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "skills");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SummarizeSkill");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "WriterSkill");

定义自己的 ASK。你想要 Kernel 去做什么呢？

In [ ]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other.";
var originalPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(JsonSerializer.Serialize(originalPlan, new JsonSerializerOptions { WriteIndented = true }));

正如你在上面的计划中所看到的，计划器已经采取了用户的要求，并将其转换成基于XML的计划，详细说明人工智能将如何去解决这个任务。

它利用了内核可用的技能，决定调用哪些函数来完成用户的要求。

计划的每一步的输出都被设置为setContextVariable，这使得它可以作为下一个技能的输入。

让我们也定义一个内联技能，让它对计划者可用。一定要给它一个函数名和技能名。

In [ ]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";
var shakespeareFunction = kernel.CreateSemanticFunction(skPrompt, "shakespeare", "ShakespeareSkill", maxTokens: 2000, temperature: 0.2, top);

让我们用新的skill来更新 ask

In [ ]:
var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. E-mail these ideas to my significant other";

var newPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(JsonSerializer.Serialize(newPlan, new JsonSerializerOptions { WriteIndented = true }));

## 执行计划器

现在我们有不同的计划器，让我们来执行他们！Kernel 能通过 `RunAsync` 来执行计划器

In [ ]:
var originalPlanResult = await originalPlan.InvokeAsync();

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.Result, 100));

现在我们执行并打印新的计划：

In [ ]:
var newPlanResult = await kernel.RunAsync(newPlan);
Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult.Result);